packages used

In [27]:
# !pip install pandas
# !pip install torch
# !pip install matplotlib

In [28]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import functools
import re

# Preprocessing

In [29]:
# input processing
trainSeqs = pd.read_csv("stanford-rna-3d-folding/train_sequences.csv")


indexSeqs = trainSeqs["sequence"].apply(list)


nucs = functools.reduce(set.union,indexSeqs.map(set).to_list()) # get all unique nucs at all positions
nucs.add("NONE")

indexSeqs = indexSeqs.apply(pd.Series).fillna("NONE")
idxToKeep = torch.tensor((indexSeqs != 'NONE').to_numpy(int),dtype=int)
print(idxToKeep)
nucToIdx = {nuc: i for i, nuc in enumerate(nucs)}


indexSeqs = indexSeqs.map(lambda nuc: nucToIdx[nuc])
indexSeqs.index = trainSeqs["target_id"]
# trainSeqs["sequence"] = trainSeqs["sequence"].
# trainSeqs = pd.get_dummies(trainSeqs, columns=["sequence"])


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [30]:
indexSeqs.head()

,0,1,2,3,4,5,6,7,8,9,...,4288,4289,4290,4291,4292,4293,4294,4295,4296,4297
target_id,,,,,,,,,,,,,,,,,,,,,
1SCL_A,5,5,5,3,5,6,3,6,4,5,...,0,0,0,0,0,0,0,0,0,0
1RNK_A,5,5,6,5,6,4,5,3,5,5,...,0,0,0,0,0,0,0,0,0,0
1RHT_A,5,5,5,4,6,3,5,4,6,5,...,0,0,0,0,0,0,0,0,0,0
1HLX_A,5,5,5,4,3,4,4,6,3,3,...,0,0,0,0,0,0,0,0,0,0
1HMH_E,5,5,6,5,4,6,6,6,3,5,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# output label processing

trainLabels = pd.read_csv("stanford-rna-3d-folding/train_labels.csv")
trainLabels["ID"] = trainLabels["ID"].map(lambda x: re.sub("_\d+$","",x))
pivotedLabels = trainLabels.pivot(index="ID",columns="resid",values=["x_1","y_1","z_1"])
pivotedLabels.fillna(0,inplace=True)
pivotedLabels.shape
labelTensor = torch.tensor(pivotedLabels.to_numpy().reshape(pivotedLabels.shape[0],3,-1)).transpose(1,2)
labelTensor = torch.nn.functional.normalize(labelTensor)
print(labelTensor[0])
pivotedLabels.head(n=1)
# print(indexSeqs.index.unique(),  trainLabels.ID.str.split(r"_(\d+)",expand=True)[0].unique())
# assert indexSeqs.index.unique() == trainLabels.ID.str.split(r"_(\d+)",expand=True)[0].unique()
# trainLabels.head()#pivot(index="ID", columns="resname", values="x_1")

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/var/folders/jd/kcpyjtsx6b7bft_5y_ps8kzm0000gn/T/ipykernel_14119/975754630.py:4: SyntaxWarning: invalid escape sequence '\d'
  trainLabels["ID"] = trainLabels["ID"].map(lambda x: re.sub("_\d+$","",x))


tensor([[ 0.3145, -0.2184, -0.2904],
        [ 0.2652, -0.2449, -0.3314],
        [ 0.2102, -0.2303, -0.2959],
        ...,
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]])


x_1                                                    \
resid        1      2       3       4       5       6          7      
ID                                                                    
17RA_A  35.856998  30.23  23.968  19.296  16.362  15.636  16.969999   

                                         ...  z_1                           \
resid        8          9          10    ... 4289 4290 4291 4292 4293 4294   
ID                                       ...                                 
17RA_A  20.391001  24.370001  26.341999  ...  0.0  0.0  0.0  0.0  0.0  0.0   

                            
resid  4295 4296 4297 4298  
ID                          
17RA_A  0.0  0.0  0.0  0.0  

[1 rows x 12894 columns]

# model architecture and training

In [37]:
class Model(nn.Module):
    def __init__(self, inputDim):
        super().__init__()
        
        self.embedding = nn.Embedding(inputDim, 512)
        encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        self.tBlock1 = nn.TransformerEncoder(encoder_layer, 2)
        self.lastLayer = nn.Linear(512,3)
        print("made model with", sum(p.numel() for p in self.parameters()), "parameters of which", sum(p.numel() for p in self.parameters() if p.requires_grad), "are trainable")

    def forward(self, input:torch.tensor):
        output = self.embedding(input)
        output = self.tBlock1(output)
        # output = output.transpose(1,2)
        outputShape = output.shape[:2] + (3,)
        output = output.view(-1,512)
        print(output.shape)

        
        output = self.lastLayer(output)

        return output.view(outputShape)

In [38]:
model = Model(len(nucs))

opter = torch.optim.Adam(model.parameters())
for _  in range(100):
    x = torch.tensor(indexSeqs.iloc[:10].values, dtype=torch.int)
    # print(x)
    loss = torch.mean((labelTensor[:10] - model(x)) ** 2 * idxToKeep.unsqueeze(-1)[:10])
    print(loss)
    loss.backward()
    opter.step()

# # TODO: understand this
# n, d, m = 3, 5, 7
# embedding = nn.Embedding(n, d, max_norm=True)
# W = torch.randn((m, d), requires_grad=True)
# idx = torch.tensor([1, 2])
# a = embedding.weight.clone() @ W.t()  # weight must be cloned for this to be differentiable
# b = embedding(idx) @ W.t()  # modifies weight in-place
# out = (a.unsqueeze(0) + b.unsqueeze(1))
# loss = out.sigmoid().prod()
# loss.backward()

made model with 6309891 parameters of which 6309891 are trainable
torch.Size([42980, 512])
tensor(0.0024, grad_fn=<MeanBackward0>)
torch.Size([42980, 512])
tensor(0.1870, grad_fn=<MeanBackward0>)
torch.Size([42980, 512])
tensor(0.0494, grad_fn=<MeanBackward0>)
torch.Size([42980, 512])
tensor(0.0031, grad_fn=<MeanBackward0>)
torch.Size([42980, 512])
tensor(0.0136, grad_fn=<MeanBackward0>)
torch.Size([42980, 512])
tensor(0.0167, grad_fn=<MeanBackward0>)
torch.Size([42980, 512])
tensor(0.0074, grad_fn=<MeanBackward0>)
torch.Size([42980, 512])
tensor(0.0053, grad_fn=<MeanBackward0>)


KeyboardInterrupt: 

In [ ]:
labelTensor

tensor([[[ 3.5857e+01, -1.0769e+01, -7.5480e+00],
         [ 3.0230e+01, -1.2075e+01, -8.6140e+00],
         [ 2.3968e+01, -1.1356e+01, -7.6900e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 1.2940e+00, -2.1080e+01, -4.4680e+00],
         [ 5.6090e+00, -2.0149e+01, -1.0550e+00],
         [ 1.1399e+01, -1.9095e+01, -4.6200e-01],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 1.2475e+01,  5.5330e+00,  4.2010e+00],
         [ 1.0106e+01,  6.6700e+00, -1.0970e+00],
         [ 7.7510e+00,  3.8470e+00, -4.7390e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        ...,

        [[ 3.1236e+01,  3.7930e+00, -6